Importation des modules

In [2]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement torch==2.0.1 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0, 2.7.0)
ERROR: No matching distribution found for torch==2.0.1


Inférence

In [ ]:
import model_loader
import pipeline
from PIL import Image
from transformers import CLIPTokenizer
import torch

DEVICE = "cpu"

ALLOW_CUDA = False
ALLOW_MPS = False

if ALLOW_CUDA and torch.cuda.is_available():
    DEVICE = "cuda"
elif ALLOW_MPS and (torch.has_mps or torch.backends.mps.is_available()):
    DEVICE = "mps"

print(f"Using device: {DEVICE}")


tokenizer = CLIPTokenizer("../data/vocab.json", merges_file = "../data/merges.txt")
model_file = "../data/v1-5-pruned-emaonly.ckpt"
models = model_loader.preload_models(model_file, DEVICE)

In [ ]:
#TEXT TO IMAGE
prompt = input("Votre prompt: ") #A cat stretching on the floor, highly detailed, ultra sharp, cinematic, 8k resolution
negative_prompt = input("Votre prompt négatif: ") #
do_cfg = True
cfg_scale = 7 #entre 1 et 14, pour dire a quel point l'IA fait attention au prompt

#IMAGE TO IMAGE
input_image = None
image_path = "../images/"#+input("Nom et extention de l'image")
#input_image = Image.open(image_path)
strength = 0.9 #combien de bruit on met dans l'image

sampler = "ddpm"
n_inference_steps = 50
seed = 42

output_image = pipeline.generate(
    prompt = prompt,
    negative_prompt = negative_prompt,
    do_cfg = do_cfg,
    cfg_scale = cfg_scale,
    sampler_name = sampler,
    models = models,
    n_inference_steps = n_inference_steps,
    seed = seed,
    device = DEVICE,
    idle_device = "cpu",
    tokenizer = tokenizer,
    input_image = input_image,
    strength = strength
)

Image.fromarray(output_image)